In [ ]:
import os
from pydantic import BaseModel, Field
from typing import Optional, List
import json
from utils import neo4j_driver

In [ ]:
class Location(BaseModel):
    address: Optional[str] = Field(..., description="The street address of the location.")
    city: Optional[str] = Field(..., description="The city of the location.")
    state: Optional[str] = Field(..., description="The state or region of the location.")
    country: str = Field(..., description="The country of the location. Use the two-letter ISO standard.")

class Organization(BaseModel):
    name: str = Field(..., description="The name of the organization.")
    location: Location = Field(..., description="The primary location of the organization.")
    role: str = Field(..., description="The role of the organization in the contract, such as 'provider', 'client', 'supplier', etc.")

contract_types = [
    "Service Agreement",
    "Licensing Agreement",
    "Non-Disclosure Agreement (NDA)",
    "Partnership Agreement",
    "Lease Agreement"
]


class Contract(BaseModel):
    contract_type: str = Field(..., description="The type of contract being entered into.", enum=contract_types)
    parties: List[Organization] = Field(..., description="List of parties involved in the contract, with details of each party's role.")
    effective_date: Optional[str] = Field(..., description="The date when the contract becomes effective. Use yyyy-MM-dd format.")
    term: str = Field(..., description="The duration of the agreement, including provisions for renewal or termination.")
    contract_scope: str = Field(..., description="Description of the scope of the contract, including rights, duties, and any limitations.")
    end_date: Optional[str] = Field(..., description="The date when the contract becomes expires. Use yyyy-MM-dd format.")
    total_amount: Optional[float] = Field(..., description="Total value of the contract.")
    governing_law: Optional[Location] = Field(..., description="The jurisdiction's laws governing the contract.")
    


In [ ]:
system_message = """
You are an expert in extracting structured information from legal documents and contracts.
Identify key details such as parties involved, dates, terms, obligations, and legal definitions.
Present the extracted information in a clear, structured format. Be concise, focusing on essential
legal content and ignoring unnecessary boilerplate language."""

In [ ]:
import ollama

def extract(document, model="llama3.2"):
    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": document}
        ],
        format=Contract.model_json_schema()
    )
    return Contract.model_validate_json(response.message.content)

In [ ]:
with open("data/license_agreement.txt") as f:
    document = f.read()

In [ ]:
data = extract(document)
data

In [ ]:
print(data.model_dump_json(indent=2))

In [ ]:
neo4j_driver.execute_query(
    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Contract) REQUIRE c.id IS UNIQUE;"
)
neo4j_driver.execute_query(
    "CREATE CONSTRAINT IF NOT EXISTS FOR (o:Organization) REQUIRE o.name IS UNIQUE;"
)
neo4j_driver.execute_query(
    "CREATE CONSTRAINT IF NOT EXISTS FOR (l:Location) REQUIRE l.fullAddress IS UNIQUE;"
)


In [ ]:
import_query = """WITH $data AS contract_data
// Create Contract node
MERGE (contract:Contract {id: randomUUID()})
SET contract.contract_type = contract_data.contract_type,
  contract.effective_date =  contract_data.effective_date,
  contract.term = contract_data.term,
  contract.contract_scope =  contract_data.contract_scope,
  contract.end_date = contract_data.end_date,
  contract.total_amount = contract_data.total_amount,
  contract.governing_law = contract_data.governing_law.state + ' ' + contract_data.governing_law.country
WITH contract, contract_data
// Create Party nodes and their locations
UNWIND contract_data.parties AS party
MERGE (p:Organization {name: party.name})
MERGE (loc:Location {
  fullAddress: party.location.address + ' ' +
                party.location.city + ' ' +
                party.location.state + ' ' +
                party.location.country})
SET loc.address = party.location.address,
  loc.city = party.location.city,
  loc.state =  party.location.state,
  loc.country = party.location.country
// Link party to their location
MERGE (p)-[:LOCATED_AT]->(loc)
// Link parties to the contract
MERGE (p)-[r:HAS_PARTY]->(contract)
SET r.role = party.role
"""
neo4j_driver.execute_query(import_query, data=data.model_dump())

In [ ]:
print(data.contract_type)